<a href="https://colab.research.google.com/github/RobolinkInc/zumi/blob/master/ZumiDrawingClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Landmark Classifier

In [0]:
# mount Google Drive for loading/uploading
# this cell only needs to be ran once per session
from google.colab import drive
drive.mount("/content/drive")

In [0]:
# import modules
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, LeakyReLU
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy
import cv2

# Setting Initial Parameters

In [0]:
# define image width and height
img_width, img_height = 64, 36

# define training and validation image directories
# directories must be arranged like so:
#
# train_data_dir/
#     classA/
#         classA0.png
#         ...
#         imageA1000.png
#     classB/
#         classB0.png
#         ...
#         classB1000.png
#
# validation_data_dir/
#     classA/
#         classA0.png
#         ...
#         imageA100.png
#     classB/
#         classB0.png
#         ...
#         classB100.png

train_data_dir = '/content/drive/My Drive/MiniTest/'
validation_data_dir = '/content/drive/My Drive/Validation/'

# define model parameters
# nb_train_samples = Sum of all images in training image directory
# nb_validation_samples = Sum of all images in validation image directory
# epochs = Number of times you want to train the model. More epochs = better accuracy (+), longer training time (-), higher risk of overfitting (--)
# batch_size = Number of images that are grouped together for analysis. (nb_train_samples + nb_validation samples) % batch_size MUST equal 0
#                 Larger batch_size = better accuracy (+), exponentially larger model size (--)

nb_train_samples = 60
nb_validation_samples = 60
epochs = 30
batch_size = 6

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# define checkpoint parameters
# currently, the weights from the most accuracte model are saved to filepath
filepath='/content/drive/My Drive/weights-12-3-18.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=True, save_best_only=True, save_weights_only=True, mode='auto', period=1)
callbacks_list = [checkpoint]

# generate a random number with a seed
# only necessary if the neural network architecture uses Dropout()
seed = 7
numpy.random.seed(seed)

# Neural Network Architecture

In [0]:
# apply each action to the model in a sequence
model = Sequential()

# these cannot be explained in one line. Use an online reference
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.6, noise_shape=None, seed=seed))
model.add(Activation('relu'))

# change x of Dense(x) to match number of classes
model.add(Dense(2))
# model.add(Dropout(0.5, noise_shape=None, seed=seed))

# if you have 2 classes, use 'sigmoid'. Otherwise, use 'softmax'
model.add(Activation('sigmoid'))

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Data Augmentation
Increase the number of training images

In [0]:
# train_datagen = imageDataGenerator(
#     rescale = normalize image
#     shear range = randomly rotate image
#     zoom range = randomly zoom/crop image
#     horizontal_flip = randomly flip the image horizontally
#     )

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Training
"acc" is the accuracy on the training data,
"val_acc" is the accuracy on the testing data

In [0]:
# generate model and graph
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
    )

print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Notes on reading the graph:
#     -For the accuracy graph, higher is better. For loss, lower is better
#     -Ideally, the lines should converge. If the lines start to diverge, that means you are underfitting/overfitting and need to either adjust
#         the model architecture or lower the number of epochs
#     -If the lines flatten out, that means that the time spent running additional epochs is being wasted

# Prediction
This prints the Neural Network's best guess of an image

In [0]:
from keras.preprocessing.image import img_to_array, load_img

# imgName = the name sans file extension
# imgSrc = the directory of the file
imgName = 'Test'
imgSrc = '/content/drive/My Drive/'

imgSrc = imgSrc + imgName + '.png'
img = load_img(imgSrc,False,target_size=(img_width,img_height))
x = img_to_array(img)
maximum = x.max()
if maximum > 0:
  scaledValue = 255.0/maximum
  x *= scaledValue
x = numpy.expand_dims(x, axis=0)
pred = model.predict(x)
print(pred)

# Notes on reading the result:
#     -Each number in the array represents the probability of the image belonging to a particular class, with a value from 0-1
#     -The classes are arranged in alphabetical order, so if you had 2 classes "cat" and "dog", "cat" would be represented by the first element